In [11]:
import numpy as np

# Calculation of thermodynamic data

This is done in the end of all qce iterations. </br>
The partition functions, volumes and populations are calculated for each molecule at each temperature. </br>
They are then used for the calculation of the thermodynamic data.

## calculation of the system partition function
Attention: This subroutine is missing the summand arising from the particle </br>
indistinguishability.

$$ Q_{\text{sys}}(T) = \prod_{i=1}^{N_{\text{clust}}} Q(T)^{N_{\text{clust}}(T)} $$
$$ \ln{Q_{\text{sys}}(T)} = \sum_{i=1}^{N_{\text{clust}}} N_{\text{clust}}(T) \ln{Q(T)} $$

In [12]:
def calc_lnq_sys(ntemp, nclust, pop, ln_clust, lnq_sys):
    # ntemp     : input     - number of temperatures at which the system is simulated
    # nclust    : input     - number of clusters in the system
    # pop       : input     - population of each cluster at each temperature
    # ln_clust  : input     - ln of the population of each cluster
    # lnq_sys   : output    - ln of the partition function of the system
    
    lnq_sys = np.zeros(ntemp)
    for i in range(ntemp):
        for j in range(nclust):
            lnq_sys[i] += pop[i][j] * ln_clust[i][j]
            
    return lnq_sys

#### Tests

In [13]:
ntemp = 5
nclust = 4

pop = np.array([[0.5, 0.3, 0.1, 0.1],
                [0.4, 0.3, 0.2, 0.1],
                [0.3, 0.3, 0.3, 0.1],
                [0.2, 0.3, 0.4, 0.1],
                [0.1, 0.3, 0.5, 0.1]])

ln_clust = np.array([[-5.2, 3.8, 2.3, 1.3],
                     [-4.2, 3.3, 2.8, 1.3],
                     [-3.2, 3.3, 3.3, 1.3],
                     [-2.2, 3.3, 4.3, 1.3],
                     [-1.2, 3.3, 5.3, 1.3]])

lnq_sys = np.zeros(ntemp)

lnq_sys = calc_lnq_sys(ntemp, nclust, pop, ln_clust, lnq_sys)
print(lnq_sys)

[-1.10000000e+00 -3.33066907e-16  1.15000000e+00  2.40000000e+00
  3.65000000e+00]
